In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset["Purchased"].value_counts()
#refer how many class classified
#to find balanced or imbalanced
#unique entry &counts---->(0-143,1-143)->balanced
#(0-257,1-143)->imbalanced

0    257
1    143
Name: Purchased, dtype: int64

In [7]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [8]:
indep.shape#400-length ,3-i/p column

(400, 3)

In [9]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [10]:
#split into training & test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)



In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [12]:
#replace regression to classifier for classification
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid={'criterion':['gini', 'entropy', 'log_loss'],'splitter':['best', 'random']}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=1,scoring='f1_weighted')
#f1_weighted suitable for all balanced &imbalance including precision,recall)
#fitting model for grid search
grid.fit(x_train,y_train)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END .....criterion=gini, splitter=best;, score=0.800 total time=   0.0s
[CV 2/5] END .....criterion=gini, splitter=best;, score=0.821 total time=   0.0s
[CV 3/5] END .....criterion=gini, splitter=best;, score=0.805 total time=   0.0s
[CV 4/5] END .....criterion=gini, splitter=best;, score=0.858 total time=   0.0s
[CV 5/5] END .....criterion=gini, splitter=best;, score=0.928 total time=   0.0s
[CV 1/5] END ...criterion=gini, splitter=random;, score=0.893 total time=   0.0s
[CV 2/5] END ...criterion=gini, splitter=random;, score=0.821 total time=   0.0s
[CV 3/5] END ...criterion=gini, splitter=random;, score=0.808 total time=   0.0s
[CV 4/5] END ...criterion=gini, splitter=random;, score=0.873 total time=   0.0s
[CV 5/5] END ...criterion=gini, splitter=random;, score=0.928 total time=   0.0s
[CV 1/5] END ..criterion=entropy, splitter=best;, score=0.819 total time=   0.0s
[CV 2/5] END ..criterion=entropy, splitter=best;,

C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\anaconda\lib\site-packages\sklearn\tree\_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\anaconda\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warnings.warn(some_fits_failed_message, FitFailedWarning)
C:\anaconda\l

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [13]:
#print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
grid_predictions=grid.predict(x_test)



In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)#optional

f1_macro value for best parameter{'criterion': 'gini', 'splitter': 'random'}: 0.9080557496700056


In [15]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [16]:
# 79,0 correct diagonal,41,0 wrong diagonal
#type1 error should low
print(cm)

[[74  5]
 [ 6 35]]


In [17]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [18]:
#overall performance(accuracy)=0.89
print(clf_report)

              precision    recall  f1-score   support

           0       0.93      0.94      0.93        79
           1       0.88      0.85      0.86        41

    accuracy                           0.91       120
   macro avg       0.90      0.90      0.90       120
weighted avg       0.91      0.91      0.91       120



In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])#receiver operating chareteristics area under curve(roc_auc)

0.8951836986724298

In [20]:
table = pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001183,0.000419,0.001179,0.000411,gini,best,"{'criterion': 'gini', 'splitter': 'best'}",0.799537,0.821429,0.804584,0.858289,0.927778,0.842323,0.047432,3
1,0.001149,0.000434,0.001204,0.000405,gini,random,"{'criterion': 'gini', 'splitter': 'random'}",0.892857,0.821429,0.807622,0.872761,0.927778,0.864489,0.044647,1
2,0.000994,0.000025,0.001396,0.000486,entropy,best,"{'criterion': 'entropy', 'splitter': 'best'}",0.819142,0.840114,0.823129,0.858289,0.927778,0.853690,0.039552,2
3,0.001050,0.000079,0.001139,0.000348,entropy,random,"{'criterion': 'entropy', 'splitter': 'random'}",0.870130,0.802399,0.806153,0.804432,0.868182,0.830259,0.031787,4
4,0.000997,0.000010,0.000000,0.000000,log_loss,best,"{'criterion': 'log_loss', 'splitter': 'best'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5,0.001279,0.000390,0.000000,0.000000,log_loss,random,"{'criterion': 'log_loss', 'splitter': 'random'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6


In [22]:
age_input=int(input("Age:"))
estimated_salary=int(input("salary:"))
gender_male_input=int(input("sex male 0 or 1:"))


Age:4
salary:34
sex male 0 or 1:0


In [23]:
Future_Prediction=grid.predict([[age_input,estimated_salary,gender_male_input]])
print("Future_Prediction{}".format(Future_Prediction))


Future_Prediction[1]


In [24]:
import pickle
filename="finalized_model_Grid.sav"
pickle.dump(grid,open(filename,'wb'))

In [25]:
loaded_model=pickle.load(open("finalized_model_Grid.sav",'rb'))
result=loaded_model.predict([[36,33000,0]])

In [26]:
result

array([1], dtype=int64)